In [1]:
import requests
import sqlite3
from datetime import date

def obtener_bitcoin():
    try:
        response = requests.get("https://api.coindesk.com/v1/bpi/currentprice.json")
        response.raise_for_status()
        return response.json()["bpi"]
    except requests.RequestException:
        print("Error al obtener el precio del Bitcoin:")
        return None

def tipo_cambio_sunat(fecha):
    try:
        url = f"https://api.apis.net.pe/v1/tipo-cambio-sunat?fecha={fecha}"
        response = requests.get(url)
        response.raise_for_status()
        return response.json()["compra"]
    except requests.RequestException:
        print("Error al obtener el tipo de cambio en dolar")
        return None
    
def guardar_bitcoin_en_basedatos():
    precio_bitcoin = obtener_bitcoin()
    
    fecha_actual = date.today().strftime('%Y-%m-%d')
    precio_dolar = tipo_cambio_sunat(fecha_actual)
    
    precio_soles = precio_bitcoin['USD']['rate_float'] * precio_dolar

    conexion = sqlite3.connect('base.db')
    cursor = conexion.cursor()

    cursor.execute('''CREATE TABLE IF NOT EXISTS bitcoin (
                        FECHA TEXT PRIMARY KEY,
                        precio_usd REAL,
                        precio_gbp REAL,
                        precio_eur REAL,
                        precio_pen REAL
                    )''')

    cursor.execute("INSERT INTO bitcoin (FECHA, precio_usd, precio_gbp, precio_eur, precio_pen) VALUES (?, ?, ?, ?, ?)",
                   (fecha_actual, precio_bitcoin['USD']['rate_float'], precio_bitcoin['GBP']['rate_float'],
                    precio_bitcoin['EUR']['rate_float'], precio_soles))

    conexion.commit()
    conexion.close()

    print("Datos guardados en la tabla 'bitcoin' de la base de datos 'base.db'")

In [2]:
guardar_bitcoin_en_basedatos()

Datos guardados en la tabla 'bitcoin' de la base de datos 'base.db'


In [3]:
import sqlite3

def calcular_compra(cant_bitcoins, moneda):
    
    conn = sqlite3.connect('base.db')
    cursor = conn.cursor()

    cursor.execute(f"SELECT PRECIO_{moneda.upper()} FROM bitcoin ORDER BY FECHA DESC LIMIT 1")
    precio_bitcoin = cursor.fetchone()[0]

    precio_compra = cant_bitcoins * precio_bitcoin

    conn.close()

    return precio_compra

precio_pen = calcular_compra(10, 'pen')
print(f"El precio de compra de 10 bitcoins en soles es: {precio_pen:.2f} soles")

precio_eur = calcular_compra(10, 'eur')
print(f"El precio de compra de 10 bitcoins en euros es: {precio_eur:.2f} euros")

El precio de compra de 10 bitcoins en soles es: 1944202.20 soles
El precio de compra de 10 bitcoins en euros es: 471522.39 euros
